## Classificador de Spam - Naive Bayes

## Importa todos os pacotes de Processamento de Linguagem Natural

In [1]:
import nltk
#nltk.download('all')
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

## Ler arquivo de dados  - SMS de celular

In [6]:
# Dados coletados e Classificados pelo Data Scientist Rodrigo Santana Ferreira 
dataset = pd.read_csv('spam.csv', encoding='latin-1')

In [7]:
# número de tuplas
print("Número de linhas: ",len(dataset))

Número de linhas:  5572


In [8]:
# número de atributos
print("Número de atributos: ",len(dataset.columns))

Número de atributos:  5


In [9]:
# atributos
dataset.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [10]:
# dataset
dataset.head(8)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN


## Limpando o dataset

In [13]:
dataset2 = dataset.loc[:, ['v1', 'v2']]
dataset2.columns = ['classe', 'texto']
dataset2.head()

,classe,texto
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
dataset2.classe.value_counts()

ham     4825
spam     747
Name: classe, dtype: int64

In [16]:
dataset2.tail()

,classe,texto
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


**Conta a quantidade de linhas de tweets neutros, positivos e negativos**

## Pre-Processamento 

** Separando tweets e suas classes
**

In [17]:
texto = dataset2['texto'].values
classes = dataset2['classe'].values

In [69]:
texto[10:]

array(["I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.",
       'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info',
       'URGENT! You have won a 1 week FREE membership in our å£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18',
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [20]:
classes[:10]

array(['ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam',
       'spam'], dtype=object)

In [96]:
texto[classes == 'ham'][10:]

array(['Eh u remember how 2 spell his name... Yes i did. He v naughty make until i v wet.',
       'Fine if thatåÕs the way u feel. ThatåÕs the way its gota b',
       'Is that seriously how you spell his name?', ...,
       'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [98]:
texto[classes == 'spam'][:10]

array(["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       "FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv",
       'WINNER!! As a valued network customer you have been selected to receivea å£900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.',
       'Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030',
       'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info',
       'URGENT! You have won a 1 week FREE membership in our å£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18',
       'XXXMobileMovieClub:

# Gerando o modelo

In [22]:
vectorizer = CountVectorizer(analyzer="word")
freq_texto = vectorizer.fit_transform(texto)
freq_texto.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## aplicando o Naive Bayes

In [23]:
modelo = MultinomialNB()
modelo.fit(freq_texto,classes)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

** Testando o modelo com algumas instâncias simples**

In [91]:
testes = ["Yes i have. So that's why u texted. Pshew...missing you so much",
          'Free entry Katty Perry Show, click here...',
          'Winner!!! are you wish to win a lottery',
          'Have a safe trip to  Russia! enjoy...',
          'Want explicit SEX in 30 secs? Ring 02073162414 now! Costs 20p/min',
          'Temporal Convolutional Nets Take Over from RNNs for NLP Predictions',
          'U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute'
         ]

In [92]:
freq_testes = vectorizer.transform(testes)

In [93]:
freq_testes.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [94]:
# Fazendo a classificação com o modelo treinado.
modelo.predict(freq_testes)

array(['ham', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam'], dtype='<U4')

In [95]:
#df = pd.DataFrame()
#df['tweet'] = testes
#df['classificacao'] = modelo.predict(freq_testes)

tweets = testes
classificacao = modelo.predict(freq_testes)

print ("Novas Instancias Classificadas...")
for i in range(len(testes)):
    print (classificacao[i], " -> ", tweets[i] )

Novas Instancias Classificadas...
ham  ->  Yes i have. So that's why u texted. Pshew...missing you so much
spam  ->  Free entry Katty Perry Show, click here...
spam  ->  Winner!!! are you wish to win a lottery
ham  ->  Have a safe trip to  Russia! enjoy...
spam  ->  Want explicit SEX in 30 secs? Ring 02073162414 now! Costs 20p/min
ham  ->  Temporal Convolutional Nets Take Over from RNNs for NLP Predictions
spam  ->  U have won the å£750 Pound prize. 2 claim is easy, call 087187272008 NOW1! Only 10p per minute


** Avaliando o modelo **

In [38]:
# Fazendo o cross validation do modelo
import numpy as np
resultados = cross_val_predict(modelo, freq_texto, classes, cv=10)

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [39]:
freq_texto.count_nonzero()

73916

In [40]:
resultados[:10]

array(['ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam',
       'spam'], dtype='<U4')

In [41]:
# Medindo a acurácia média do modelo
metrics.accuracy_score(classes,resultados)

0.9811557788944724

### Matriz de Confusão

In [42]:
# Matriz de confusão
print (pd.crosstab(classes, resultados, rownames=['REAL'], colnames=['PREDITO'], margins=True))

PREDITO   ham  spam   All
REAL                     
ham      4760    65  4825
spam       40   707   747
All      4800   772  5572


In [43]:
# Medidas de validação do modelo
classe=['ham','spam']
print (metrics.classification_report(classes,resultados,classe))

             precision    recall  f1-score   support

        ham       0.99      0.99      0.99      4825
       spam       0.92      0.95      0.93       747

avg / total       0.98      0.98      0.98      5572



---